In [13]:
import numpy as np
import sys
import os
# Add the src folder to Python's module search path

# Navigate to the `src` folder
os.chdir('/Users/haoyuluo/Desktop/Columbia_MS/EE/ELENE6885_001_2024_3 - TOPICS IN SIGNAL PROCESSING/Final_Project/MARL-with-Rock-Paper-Scissor/src')

print("Current working directory:", os.getcwd())
from  my_gym import Config, Environment
import tensorflow as tf
import random
from collections import deque

Current working directory: /Users/haoyuluo/Desktop/Columbia_MS/EE/ELENE6885_001_2024_3 - TOPICS IN SIGNAL PROCESSING/Final_Project/MARL-with-Rock-Paper-Scissor/src


ModuleNotFoundError: No module named 'my_env'

In [2]:
# DQN for Agent Type 0
class DQNType0(tf.keras.Model):
    def __init__(self, input_dim, output_dim):
        super(DQNType0, self).__init__()
        self.fc1 = tf.keras.layers.Dense(128, activation='relu', input_shape=(input_dim,))
        self.fc2 = tf.keras.layers.Dense(128, activation='relu')
        self.fc3 = tf.keras.layers.Dense(output_dim)

    def call(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        return self.fc3(x)

# DQN for Agent Type 1
class DQNType1(tf.keras.Model):
    def __init__(self, input_dim, output_dim):
        super(DQNType1, self).__init__()
        self.fc1 = tf.keras.layers.Dense(128, activation='relu', input_shape=(input_dim,))
        self.fc2 = tf.keras.layers.Dense(128, activation='relu')
        self.fc3 = tf.keras.layers.Dense(output_dim)

    def call(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        return self.fc3(x)

In [3]:
# Agent Class for Type 0
class AgentType0:
    def __init__(self, env):
        self.env = env
        self.grid_size = env.grid_size
        self.n_actions = 4  # Up, Down, Left, Right

        self.input_dim = self.grid_size * self.grid_size  # Flattened grid

        # DQN networks
        self.q_network = DQNType0(input_dim=self.input_dim, output_dim=self.n_actions)
        self.target_network = DQNType0(input_dim=self.input_dim, output_dim=self.n_actions)
        self.target_network.set_weights(self.q_network.get_weights())

        # Optimizer and loss
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
        self.loss_fn = tf.keras.losses.MeanSquaredError()

        # Replay buffer
        self.memory = deque(maxlen=10000)
        self.gamma = 0.99
        self.batch_size = 64
        self.epsilon = 1.0
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01

    def select_action(self, state):
        if random.random() < self.epsilon:
            return random.randint(0, self.n_actions - 1)
        else:
            state = np.expand_dims(state, axis=0).astype(np.float32)
            q_values = self.q_network(state)
            return np.argmax(q_values.numpy()[0])

    def store_transition(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    # Idea similar to 1predator_1prey.update_q_network
    def update_q_network(self):
        if len(self.memory) < self.batch_size:
            return
        batch = random.sample(self.memory, self.batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)

        states = np.array(states).astype(np.float32)
        actions = np.array(actions)
        rewards = np.array(rewards).astype(np.float32)
        next_states = np.array(next_states).astype(np.float32)
        dones = np.array(dones).astype(np.float32)

        # Compute target Q-values
        next_q_values = self.target_network(next_states)
        max_next_q_values = np.max(next_q_values.numpy(), axis=1)
        target_q_values = rewards + (1 - dones) * self.gamma * max_next_q_values

        # Compute current Q-values
        with tf.GradientTape() as tape:
            q_values = self.q_network(states)
            indices = np.arange(self.batch_size)
            action_q_values = tf.gather_nd(q_values, np.vstack([indices, actions]).T)
            loss = self.loss_fn(target_q_values, action_q_values)

        # Update network parameters
        grads = tape.gradient(loss, self.q_network.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.q_network.trainable_variables))

    def update_target_network(self):
        self.target_network.set_weights(self.q_network.get_weights())
        
    # Agent Class for Type 1 (similar to Type 0)
class AgentType1:
    def __init__(self, env):
        self.env = env
        self.grid_size = env.grid_size
        self.n_actions = 4  # Up, Down, Left, Right

        self.input_dim = self.grid_size * self.grid_size  # Flattened grid

        # DQN networks
        self.q_network = DQNType1(input_dim=self.input_dim, output_dim=self.n_actions)
        self.target_network = DQNType1(input_dim=self.input_dim, output_dim=self.n_actions)
        self.target_network.set_weights(self.q_network.get_weights())

        # Optimizer and loss
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
        self.loss_fn = tf.keras.losses.MeanSquaredError()

        # Replay buffer
        self.memory = deque(maxlen=10000)
        self.gamma = 0.99
        self.batch_size = 64
        self.epsilon = 1.0
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01

    def select_action(self, state):
        if random.random() < self.epsilon:
            return random.randint(0, self.n_actions - 1)
        else:
            state = np.expand_dims(state, axis=0).astype(np.float32)
            q_values = self.q_network(state)
            return np.argmax(q_values.numpy()[0])

    def store_transition(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def update_q_network(self):
        if len(self.memory) < self.batch_size:
            return
        batch = random.sample(self.memory, self.batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)

        states = np.array(states).astype(np.float32)
        actions = np.array(actions)
        rewards = np.array(rewards).astype(np.float32)
        next_states = np.array(next_states).astype(np.float32)
        dones = np.array(dones).astype(np.float32)

        # Compute target Q-values
        next_q_values = self.target_network(next_states)
        max_next_q_values = np.max(next_q_values.numpy(), axis=1)
        target_q_values = rewards + (1 - dones) * self.gamma * max_next_q_values

        # Compute current Q-values
        with tf.GradientTape() as tape:
            q_values = self.q_network(states)
            indices = np.arange(self.batch_size)
            action_q_values = tf.gather_nd(q_values, np.vstack([indices, actions]).T)
            loss = self.loss_fn(target_q_values, action_q_values)

        # Update network parameters
        grads = tape.gradient(loss, self.q_network.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.q_network.trainable_variables))

    def update_target_network(self):
        self.target_network.set_weights(self.q_network.get_weights())
        
def get_agent_state(env, agent_id):
    # Use the full grid as the observation
    grid = env.grid.copy()
    state = grid.flatten()
    return state

In [4]:


if __name__ == "__main__":
    config_file = "config.yaml"
    config = Config(config_file)
    env = Environment(config)

    # Load configuration
    config = Config("mypackage/config.yaml")  # Path to your YAML file
    env = Environment(config)  # Initialize the environment

    # Number of steps in one episode
    # max_steps = 50  # Adjust based on the problem

    # Run one episode
    print("Initial Environment:")
    env.render()
    
# Initialize agents
    agent_type0 = AgentType0(env)
    agent_type1 = AgentType1(env)

    # Map agent IDs to their types
    agent_id_to_type = {}
    for idx, pos in enumerate(env.positions):
        agent_id_to_type[idx] = pos['agent']

    num_episodes = 1000
    target_update_freq = 10
    max_steps_per_episode = 100

for episode in range(num_episodes):
    env.reset()
    env.render()
    total_rewards = {0: 0, 1: 0}
    done = False
    timestep = 0

    while not done and timestep < max_steps_per_episode:
        actions = []
        states = []  # List to store states before actions
        # Each agent selects an action
        for agent_id in range(env.total_agents):
            if agent_id not in env.captured_agents:
                agent_type = env.positions[agent_id]['agent']
                state = get_agent_state(env, agent_id)
                states.append(state)  # Store the state before action
                if agent_type == 0:
                    action = agent_type0.select_action(state)
                elif agent_type == 1:
                    action = agent_type1.select_action(state)
                else:
                    action = random.randint(0, 3)  # Default random action
            else:
                action = None  # Agent has been captured
                states.append(None)  # Placeholder for consistency
            actions.append(action)

        # Environment processes all actions
        env.step(actions, timestep)
        timestep += 1

        # Update each agent
        for agent_id in range(env.total_agents):
            if agent_id not in env.captured_agents:
                agent_type = env.positions[agent_id]['agent']
                state = states[agent_id]  # State before action
                reward = env.reward[agent_type]
                done = env.done
                next_state = get_agent_state(env, agent_id)  # State after action

                if agent_type == 0:
                    agent_type0.store_transition(state, actions[agent_id], reward, next_state, done)
                    agent_type0.update_q_network()
                    total_rewards[0] += reward
                elif agent_type == 1:
                    agent_type1.store_transition(state, actions[agent_id], reward, next_state, done)
                    agent_type1.update_q_network()
                    total_rewards[1] += reward

        if env.done:
            break

    # Update target networks and decay epsilon
    if episode % target_update_freq == 0:
        agent_type0.update_target_network()
        agent_type1.update_target_network()

    if agent_type0.epsilon > agent_type0.epsilon_min:
        agent_type0.epsilon *= agent_type0.epsilon_decay
    if agent_type1.epsilon > agent_type1.epsilon_min:
        agent_type1.epsilon *= agent_type1.epsilon_decay

    print(f"Episode {episode + 1}/{num_episodes}, Total Rewards: {total_rewards}")


    # for step in range(max_steps):
    #     if env.done:
    #         print(f"Episode ended after {step} steps.")
    #         break

    #     # Select random actions for all agents
    #     action_list = random_action_selector(env.total_agents)
        
    #     # Take a step in the environment
    #     env.step(action_list)
        
    #     # Render the environment
    #     print(f"Step {step + 1}:")

    # Check results
    # if env.done:
    #     print(f"Winner: Agent Type {env.winner}")
    # else:
    #     print("Episode ended without a winner.")

NameError: name 'Config' is not defined